## 读取数据
此单元格最后的输出为test_X和test_y, 其中test_X包含两列 ['text', 'embed']，test_y包含一列 ['label']

In [3]:
data = pd.read_json("random_pick.json")

#选取sentence包含“体育”的数据，label为1，选取sentence包含“政治”的数据，label为0，选取sentence包含”天气“的数据，label为2, 选取sentence包含“音乐”的数据，label为3
sel_data = data[(data["sentence"].str.contains("体育")) | (data["sentence"].str.contains("政治")) | (data["sentence"].str.contains("天气")) | (data["sentence"].str.contains("音乐"))]
sel_data = sel_data.reset_index(drop=True)
sel_data.loc[sel_data["sentence"].str.contains("体育"), "label"] = 1
sel_data.loc[sel_data["sentence"].str.contains("政治"), "label"] = 0
sel_data.loc[sel_data["sentence"].str.contains("天气"), "label"] = 2
sel_data.loc[sel_data["sentence"].str.contains("音乐"), "label"] = 3

# reset the index
sel_data = sel_data.reset_index(drop=True)

# transform the left_embed and right_embed into an numpy array
left_embed = np.array(sel_data["left_embed"].tolist())
right_embed = np.array(sel_data["right_embed"].tolist())

# concatenate the left_embed and right_embed
embed = np.concatenate((left_embed, right_embed), axis=1)
sel_data["embed"] = embed.tolist()

test_X = sel_data[['sentence', 'embed']]
test_y = sel_data[['label']]
test_X.columns = ['text', 'embed']
test_y.columns = ['label']

## 绘图

In [6]:
from tsne import TSNE_Plot
# 包含label
tsne_plot = TSNE_Plot(test_X['text'], test_X['embed'], test_y['label'])
tsne_plot.tsne_plot()

--------------------------------------------------------------------------------
TSNE(early_exaggeration=12, metric='cosine', n_jobs=8, random_state=42,
     verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using exact search using cosine distance...
   --> Time elapsed: 0.13 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.01 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.02 seconds
===> Running optimization with exaggeration=12.00, lr=18.42 for 250 iterations...
Iteration   50, KL divergence 1.4449, 50 iterations in 0.1034 sec
Iteration  100, KL divergence 1.4957, 50 iterations in 0.0563 sec
Iteration  150, KL divergence 1.4958, 50 iterations in 0.0524 sec
Iteration  200, KL divergence 1.4957, 50 iterations in 0.0540 sec
Iteration  250, KL divergence 1.4958, 50 iterations in 0.1062 sec
   --> Time elapsed: 0.37 seconds
===> Running optimization with exaggerat

d:\14183\Desktop\LLM\Embedding\tsne.py:181: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

d:\14183\Desktop\LLM\Embedding\tsne.py:182: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [7]:
# 不包含label
tsne_plot = TSNE_Plot(test_X['text'], test_X['embed'], N = 4)
tsne_plot.tsne_plot()

--------------------------------------------------------------------------------
TSNE(early_exaggeration=12, metric='cosine', n_jobs=8, random_state=42,
     verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using exact search using cosine distance...
   --> Time elapsed: 0.14 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.01 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.01 seconds
===> Running optimization with exaggeration=12.00, lr=18.42 for 250 iterations...
Iteration   50, KL divergence 1.4551, 50 iterations in 0.0910 sec
Iteration  100, KL divergence 1.5002, 50 iterations in 0.0539 sec
Iteration  150, KL divergence 1.5009, 50 iterations in 0.0512 sec
Iteration  200, KL divergence 1.5009, 50 iterations in 0.0502 sec
Iteration  250, KL divergence 1.5013, 50 iterations in 0.0498 sec
   --> Time elapsed: 0.30 seconds
===> Running optimization with exaggerat

d:\14183\Desktop\LLM\Embedding\tsne.py:181: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

d:\14183\Desktop\LLM\Embedding\tsne.py:182: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

